In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import pandas as pd
import preprocessing
from datetime import timedelta 

In [2]:
def Replace_Outliers(l1):
    l1 = pd.DataFrame({'col': l1})
    Q1 = np.percentile(l1, 25, interpolation='midpoint')
    Q3 = np.percentile(l1, 75, interpolation='midpoint')

    IQR = Q3 - Q1
    df2 = pd.DataFrame(l1)
    outliers = l1[((l1 < (Q1 - 1.5 * IQR)) | (l1 > (Q3 + 1.5 * IQR)))]
    outliers.dropna(inplace=True, axis=0)
    # df=outliers

    outliers = list(outliers.values)
    rep = np.median(l1)
    df2[df2.columns[0]] = df2[df2.columns[0]].replace(outliers, rep)

    #   return list(outliers.flatten())
    return list(df2.values.flatten())


In [3]:
def create_dataset(data, look_back=1):
    data_X, data_y = [], []
    for i in range(len(data) - look_back):
        a = data[i:(i + look_back), :]
        data_X.append(a)
        data_y.append(data[i + look_back, :])
    return np.array(data_X), np.array(data_y)

In [4]:
def forecasting(data,col,datecol):
  data[datecol] = pd.to_datetime(data[datecol])
  data=data.groupby([datecol]).sum([col]).reset_index()
  tempdate=list(data[datecol].values)
  datadup=data
  data.drop(columns=datecol,inplace=True)
  # Scale the data using MinMaxScaler
  scaler = MinMaxScaler(feature_range=(0, 1))
  data_scaled = scaler.fit_transform(data)
  train_size = int(len(data_scaled) * 0.8)
  test_size = len(data_scaled) - train_size
  train, test = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]
  look_back = 1
  train_X, train_y = create_dataset(train, look_back)
  test_X, test_y = create_dataset(test, look_back)
  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
  test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
  model = Sequential()
  model.add(LSTM(50, input_shape=(1, look_back)))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=0)
  # Make predictions on the test data
  predictions = model.predict(test_X)
  predictions = scaler.inverse_transform(predictions)
  test_y = scaler.inverse_transform(test_y)
  tomorrow_sales = model.predict(np.array([test_X[-1]]))
  tomorrow_sales = scaler.inverse_transform(tomorrow_sales)
  nextdate=[]
  datadup[datecol]=tempdate
  for i in range(0,len(tomorrow_sales)):
    nextdate.append(max(datadup[datecol]) +pd.DateOffset(i+1))
  tempdf=pd.DataFrame()
  tempdf[datecol]=nextdate
  tempdf[col]=tomorrow_sales
  tempdf=pd.concat([datadup[[datecol,col]],tempdf],ignore_index=True)



  return tempdf

In [5]:
data=pd.read_excel('/content/cona_sample.xlsx')
data

,inv_invoice_date,inv_itemwise_value_reporting_amount
0,2021-04-06,1330.92
1,2021-04-06,3363.60
2,2022-04-04,579.60
3,2022-04-04,1464.48
4,2022-04-04,1464.48
...,...,...
683422,2021-12-31,849.80
683423,2021-12-31,974.60
683424,2022-11-18,10374.00
683425,2022-11-18,-600.03


In [ ]:
forecasting(data,'inv_itemwise_value_reporting_amount','inv_invoice_date')

1/1 [==============================] - 0s 25ms/step


,inv_invoice_date,inv_itemwise_value_reporting_amount
0,2021-04-06,3162168.75
1,2021-04-07,3868091.26
2,2021-04-08,3653882.06
3,2021-04-09,4203256.48
4,2021-04-10,1985554.68
...,...,...
608,2023-01-28,12572429.82
609,2023-01-29,109064.67
610,2023-01-30,17196220.72
611,2023-01-31,26365662.48


In [ ]:
max(data.inv_invoice_date) +pd.DateOffset(1)

Timestamp('2023-02-01 00:00:00')

In [ ]:
ja={
    "school_name": ["local primary school","1"],
    "class": ["Year 1","5"]

}

In [ ]:
pd.DataFrame(ja,index=[0])

ValueError: ignored

In [ ]:
jsonStr = '''{"Index0":{"Courses": "Pandas","Discount": "1200"},
           "Index1":{"Courses": "Hadoop","Discount": "1500"},
           "Index2":{"Courses": "Spark","Discount": "1800"}
          }'''
# Convert JSON to DataFrame Using read_json()
df2 = pd.read_json(jsonStr, orient ='index')
df2

,Courses,Discount
Index0,Pandas,1200
Index1,Hadoop,1500
Index2,Spark,1800


In [10]:
data.to_dict('list')

{'inv_invoice_date': ['2021-04-06',
  '2021-04-06',
  '2022-04-04',
  '2022-04-04',
  '2022-04-04',
  '2022-04-04',
  '2022-04-04',
  '2022-04-04',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2021-04-07',
  '2022-04-12',
  '2022-04-12',
  '2022-04-12',
  '2022-04-12',
  '2022-04-12',
  '2022-04-12',
  '2021-06-12',
  '2

In [8]:
data.inv_invoice_date=data.inv_invoice_date.astype(str)

In [11]:

import json

In [20]:
with open("sample.json", "w") as outfile:
    json.dump(data.to_dict('list'), outfile)

In [16]:
def forecasting(data,col,datecol):
  data=pd.DataFrame(data)
  data[datecol] = pd.to_datetime(data[datecol])
#  data[datecol]=data[datecol].astype(str)
  data=data.groupby([datecol]).sum([col]).reset_index()
  tempdate=list(data[datecol].values)
  datadup=data
  data.drop(columns=datecol,inplace=True)
  # Scale the data using MinMaxScaler
  scaler = MinMaxScaler(feature_range=(0, 1))
  data_scaled = scaler.fit_transform(data)
  train_size = int(len(data_scaled) * 0.8)
  test_size = len(data_scaled) - train_size
  train, test = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]
  look_back = 1
  train_X, train_y = create_dataset(train, look_back)
  test_X, test_y = create_dataset(test, look_back)
  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
  test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
  model = Sequential()
  model.add(LSTM(50, input_shape=(1, look_back)))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=0)
  # Make predictions on the test data
  predictions = model.predict(test_X)
  predictions = scaler.inverse_transform(predictions)
  test_y = scaler.inverse_transform(test_y)
  tomorrow_sales = model.predict(np.array([test_X[-1]]))
  tomorrow_sales = scaler.inverse_transform(tomorrow_sales)
  nextdate=[]
  datadup[datecol]=tempdate
  for i in range(0,len(tomorrow_sales)):
    nextdate.append(max(datadup[datecol]) +pd.DateOffset(i+1))
  tempdf=pd.DataFrame()
  tempdf[datecol]=nextdate
  tempdf[col]=tomorrow_sales
  tempdf=pd.concat([datadup[[datecol,col]],tempdf],ignore_index=True)
  tempdf[datecol]=tempdf[datecol].astype(str)

  return tempdf



In [18]:
forecasting(data,'inv_itemwise_value_reporting_amount','inv_invoice_date').to_dict('list')

1/1 [==============================] - 0s 18ms/step


{'inv_invoice_date': ['2021-04-06',
  '2021-04-07',
  '2021-04-08',
  '2021-04-09',
  '2021-04-10',
  '2021-04-12',
  '2021-04-13',
  '2021-04-14',
  '2021-04-15',
  '2021-04-16',
  '2021-04-17',
  '2021-04-18',
  '2021-04-19',
  '2021-04-20',
  '2021-04-21',
  '2021-04-22',
  '2021-04-23',
  '2021-04-24',
  '2021-04-25',
  '2021-04-26',
  '2021-04-27',
  '2021-04-28',
  '2021-04-29',
  '2021-04-30',
  '2021-05-04',
  '2021-05-05',
  '2021-05-06',
  '2021-05-07',
  '2021-05-08',
  '2021-05-09',
  '2021-05-10',
  '2021-05-11',
  '2021-05-12',
  '2021-05-13',
  '2021-05-14',
  '2021-05-15',
  '2021-05-16',
  '2021-05-17',
  '2021-05-18',
  '2021-05-19',
  '2021-05-20',
  '2021-05-21',
  '2021-05-22',
  '2021-05-23',
  '2021-05-24',
  '2021-05-25',
  '2021-05-26',
  '2021-05-27',
  '2021-05-28',
  '2021-05-29',
  '2021-05-30',
  '2021-05-31',
  '2021-06-04',
  '2021-06-05',
  '2021-06-06',
  '2021-06-07',
  '2021-06-08',
  '2021-06-09',
  '2021-06-10',
  '2021-06-11',
  '2021-06-12',
  '2